In [1]:
import os
import pandas as pd
import geopandas as gpd
from shapely.geometry import Point, MultiPoint #, Polygon, MultiPolygon
from shapely.ops import nearest_points, unary_union # função para pegar o ponto vizinho mais próximo
from shapely import wkt
import folium as fl
import matplotlib.pyplot as plt
import numpy as np
import math
import glob
import json

In [6]:
esgoto_pb_2020 = pd.read_excel('./data/esgoto_pb_filtrado.xlsx')

C:\Users\miguel\AppData\Local\Temp/ipykernel_16940/935451993.py:1: FutureWarning: Your version of xlrd is 1.2.0. In xlrd >= 2.0, only the xls format is supported. As a result, the openpyxl engine will be used if it is installed and the engine argument is not specified. Install openpyxl instead.
  esgoto_pb_2020 = pd.read_excel('./data/esgoto_pb_filtrado.xlsx')


In [7]:
esgoto_pb_2020.head()

,Código do Município,Município,Estado,Ano de Referência,Código do Prestador,Prestador,Sigla do Prestador,Abrangência,Tipo de serviço,Natureza jurídica,...,ES006 - Volume de esgotos tratado,ES007 - Volume de esgotos faturado,ES008 - Quantidade de economias residenciais ativas de esgotos,ES009 - Quantidade de ligações totais de esgotos,ES012 - Volume de esgoto bruto exportado,ES013 - Volume de esgotos bruto importado,ES014 - Volume de esgoto importado tratado nas instalações do importador,ES015 - Volume de esgoto bruto exportado tratado nas instalações do importador,ES026 - População urbana atendida com esgotamento sanitário,ES028 - Consumo total de energia elétrica nos sistemas de esgotos
0,250010,Água Branca,PB,2020,25001011,Prefeitura Municipal de Água Branca,PMAB,Local,Esgotos,Administração pública direta,...,0.00,0.00,NaN,298,0.0,0.0,0,0,3800,NaN
1,250030,Alagoa Grande,PB,2020,25075000,Companhia de Águas e Esgotos da Paraíba,CAGEPA,Regional,Água e Esgoto,Sociedade de economia mista com administração ...,...,304.99,384.75,2450.0,2954,0.0,0.0,0,0,8355,113.74
2,250060,Alhandra,PB,2020,25075000,Companhia de Águas e Esgotos da Paraíba,CAGEPA,Regional,Água e Esgoto,Sociedade de economia mista com administração ...,...,58.87,72.11,396.0,652,0.0,0.0,0,0,1394,4.12
3,250073,Amparo,PB,2020,25007311,Prefeitura Municipal de Amparo,PMA,Local,Esgotos,Administração pública direta,...,0.00,0.00,NaN,400,0.0,0.0,0,0,1121,NaN
4,250077,Aparecida,PB,2020,25007711,Prefeitura Municipal de Aparecida,PMA,Local,Esgotos,Administração pública direta,...,500.00,0.00,1000.0,80,500.0,9000.0,0,1,1500,500.00


## Limites Municipios PB

In [8]:
limites_pb = gpd.read_file('https://raw.githubusercontent.com/CEARDados/mmp/master/Dados/PB_munic%C3%ADpios.json')
limites_pb.head()

,OBJECTID,GEOCODIG_M,UF,Sigla,Nome_Munic,Região,Mesorregiã,Nome_Meso,Microrregi,Nome_Micro,Shape_Leng,Shape_Area,geometry
0,1,2500106.0,25,PB,Água Branca,Nordeste,2501,Sertão Paraibano,25007,Serra do Teixeira,0.729661,0.019337,"POLYGON ((-37.70376 -7.37318, -37.69232 -7.378..."
1,2,2500205.0,25,PB,Aguiar,Nordeste,2501,Sertão Paraibano,25005,Piancó,0.969734,0.028152,"POLYGON ((-38.20504 -7.00981, -38.15629 -7.009..."
2,3,2500304.0,25,PB,Alagoa Grande,Nordeste,2503,Agreste Paraibano,25015,Brejo Paraibano,0.789787,0.026308,"POLYGON ((-35.53465 -7.00937, -35.52336 -7.031..."
3,4,2500403.0,25,PB,Alagoa Nova,Nordeste,2503,Agreste Paraibano,25015,Brejo Paraibano,0.500304,0.009923,"POLYGON ((-35.79566 -7.02333, -35.78838 -7.029..."
4,5,2500502.0,25,PB,Alagoinha,Nordeste,2503,Agreste Paraibano,25016,Guarabira,0.444048,0.007825,"POLYGON ((-35.56345 -6.90760, -35.53411 -6.928..."


In [17]:
aux = pd.DataFrame(limites_pb)
aux['Prestador'] = None
aux2 = list()
i=0

for row_pb in aux.iterrows():
    
    nome_munic = row_pb[1]['Nome_Munic']

    for row in esgoto_pb_2020.iterrows():

        if row[1]['Município'] == nome_munic:

            aux.loc[i,'Prestador'] = row[1]['Prestador']
            # i+=1
            break

    if not aux.loc[i,'Prestador']:
        aux.loc[i,'Prestador'] = 'Sem informações'
        i+=1
    else:
        i+=1

aux.head()

,OBJECTID,GEOCODIG_M,UF,Sigla,Nome_Munic,Região,Mesorregiã,Nome_Meso,Microrregi,Nome_Micro,Shape_Leng,Shape_Area,geometry,Prestador
0,1,2500106.0,25,PB,Água Branca,Nordeste,2501,Sertão Paraibano,25007,Serra do Teixeira,0.729661,0.019337,"POLYGON ((-37.70376 -7.37318, -37.69232 -7.378...",Prefeitura Municipal de Água Branca
1,2,2500205.0,25,PB,Aguiar,Nordeste,2501,Sertão Paraibano,25005,Piancó,0.969734,0.028152,"POLYGON ((-38.20504 -7.00981, -38.15629 -7.009...",Sem informações
2,3,2500304.0,25,PB,Alagoa Grande,Nordeste,2503,Agreste Paraibano,25015,Brejo Paraibano,0.789787,0.026308,"POLYGON ((-35.53465 -7.00937, -35.52336 -7.031...",Companhia de Águas e Esgotos da Paraíba
3,4,2500403.0,25,PB,Alagoa Nova,Nordeste,2503,Agreste Paraibano,25015,Brejo Paraibano,0.500304,0.009923,"POLYGON ((-35.79566 -7.02333, -35.78838 -7.029...",Sem informações
4,5,2500502.0,25,PB,Alagoinha,Nordeste,2503,Agreste Paraibano,25016,Guarabira,0.444048,0.007825,"POLYGON ((-35.56345 -6.90760, -35.53411 -6.928...",Sem informações


In [18]:
os.mkdir('dados_finalizados')
aux.to_csv('./dados_finalizados/municipios_pb_prestadores.csv')

In [21]:
cidades_atendidas_pela_CAGEPA = aux[aux['Prestador'] == 'Companhia de Águas e Esgotos da Paraíba']
cidades_atendidas_pela_CAGEPA.reset_index(inplace=True)
cidades_atendidas_pela_CAGEPA.drop(['index'],axis=1,inplace=True)
cidades_atendidas_pela_CAGEPA.head(2)
cidades_atendidas_pela_CAGEPA.to_csv('./dados_finalizados/municipios_prestador_CAGEPA.csv')

c:\Users\miguel\anaconda3\envs\CRESESBenv\lib\site-packages\pandas\core\frame.py:4308: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().drop(


In [24]:
cidades_nao_atendidas_pela_CAGEPA = aux[aux['Prestador'] != 'Companhia de Águas e Esgotos da Paraíba']
cidades_nao_atendidas_pela_CAGEPA = cidades_nao_atendidas_pela_CAGEPA[cidades_nao_atendidas_pela_CAGEPA['Prestador'] != None]
cidades_nao_atendidas_pela_CAGEPA.reset_index(inplace=True)
cidades_nao_atendidas_pela_CAGEPA.drop(['index'],axis=1,inplace=True)
cidades_nao_atendidas_pela_CAGEPA.head(2)
cidades_nao_atendidas_pela_CAGEPA.to_csv('./dados_finalizados/municipios_prestador_prefeitura.csv')

In [30]:
prefeituras = list()

for prestador in aux['Prestador']:
    if str(prestador).startswith('Prefeitura'):
        prefeituras.append(prestador)


cidades_atendidas_pela_PREFEITURAS = aux.query(f"'Prestador' in {str(prefeituras)}")

# cidades_nao_atendidas_pela_CAGEPA = aux[aux['Prestador'] != 'Companhia de Águas e Esgotos da Paraíba']
# cidades_nao_atendidas_pela_CAGEPA = cidades_nao_atendidas_pela_CAGEPA[cidades_nao_atendidas_pela_CAGEPA['Prestador'] != None]
# cidades_nao_atendidas_pela_CAGEPA.reset_index(inplace=True)
# cidades_nao_atendidas_pela_CAGEPA.drop(['index'],axis=1,inplace=True)
cidades_atendidas_pela_PREFEITURAS.head(2)
# cidades_atendidas_pela_PREFEITURAS.to_csv('./dados_finalizados/municipios_prestador_prefeitura.csv')

KeyError: False

In [10]:
dados_IDH = pd.read_excel('./data/IDH.xlsx',skiprows = 2)
dados_IDH.head()

C:\Users\miguel\AppData\Local\Temp/ipykernel_16940/450252878.py:1: FutureWarning: Your version of xlrd is 1.2.0. In xlrd >= 2.0, only the xls format is supported. As a result, the openpyxl engine will be used if it is installed and the engine argument is not specified. Install openpyxl instead.
  dados_IDH = pd.read_excel('./data/IDH.xlsx',skiprows = 2)


,Município [-],Código [-],Gentílico [-],Prefeito [2021],Área Territorial - km² [2021],População estimada - pessoas [2021],Densidade demográfica - hab/km² [2010],Escolarização <span>6 a 14 anos</span> - % [2010],IDHM <span>Índice de desenvolvimento humano municipal</span> [2010],Mortalidade infantil - óbitos por mil nascidos vivos [2020],Receitas realizadas - R$ (×1000) [2017],Despesas empenhadas - R$ (×1000) [2017],PIB per capita - R$ [2019]
0,Água Branca,2500106.0,agua branquense,EVERTON FIRMINO BATISTA,241.662,10375.0,39.94,97.6,0.572,23.26,24636.41005,23228.10166,8074.16
1,Aguiar,2500205.0,aguiarense,MANOEL BATISTA GUEDES FILHO,351.607,5622.0,16.04,98.9,0.597,-,18851.68017,18417.05179,8897.54
2,Alagoa Grande,2500304.0,alagoa-grandense,ANTÔNIO DA SILVA SOBRINHO,322.071,28384.0,88.84,97.4,0.582,12.2,50172.09386,49154.32307,9699.38
3,Alagoa Nova,2500403.0,alagoa-novense,FRANCINILDO PIMENTEL DA SILVA,128.230,20992.0,160.98,97.9,0.576,15.02,41347.83650,40184.92284,15631.55
4,Alagoinha,2500502.0,alagoinhense,MARIA RODRIGUES DE ALMEIDA FARIAS,111.361,14629.0,139.99,98.7,0.595,-,36867.87533,32853.50294,7934.57


In [11]:
dados_IDH.drop([i for i in range(223,238)],axis=0,inplace=True)

In [38]:
# dados_IDH.drop([237],axis=0,inplace=True)

In [12]:
dados_IDH

,Município [-],Código [-],Gentílico [-],Prefeito [2021],Área Territorial - km² [2021],População estimada - pessoas [2021],Densidade demográfica - hab/km² [2010],Escolarização <span>6 a 14 anos</span> - % [2010],IDHM <span>Índice de desenvolvimento humano municipal</span> [2010],Mortalidade infantil - óbitos por mil nascidos vivos [2020],Receitas realizadas - R$ (×1000) [2017],Despesas empenhadas - R$ (×1000) [2017],PIB per capita - R$ [2019]
0,Água Branca,2500106.0,agua branquense,EVERTON FIRMINO BATISTA,241.662,10375.0,39.94,97.6,0.572,23.26,24636.41005,23228.10166,8074.16
1,Aguiar,2500205.0,aguiarense,MANOEL BATISTA GUEDES FILHO,351.607,5622.0,16.04,98.9,0.597,-,18851.68017,18417.05179,8897.54
2,Alagoa Grande,2500304.0,alagoa-grandense,ANTÔNIO DA SILVA SOBRINHO,322.071,28384.0,88.84,97.4,0.582,12.2,50172.09386,49154.32307,9699.38
3,Alagoa Nova,2500403.0,alagoa-novense,FRANCINILDO PIMENTEL DA SILVA,128.230,20992.0,160.98,97.9,0.576,15.02,41347.83650,40184.92284,15631.55
4,Alagoinha,2500502.0,alagoinhense,MARIA RODRIGUES DE ALMEIDA FARIAS,111.361,14629.0,139.99,98.7,0.595,-,36867.87533,32853.50294,7934.57
...,...,...,...,...,...,...,...,...,...,...,...,...,...
218,Uiraúna,2516904.0,uiraunense,MARIA SULENE DANTAS SARMENTO,293.182,15356.0,49.52,97.8,0.636,15.96,34874.85769,33092.71892,11427.53
219,Umbuzeiro,2517001.0,umbuzeirense,JOSÉ NIVALDO DE ARAÚJO,185.578,9914.0,51.28,99.4,0.584,36.76,19713.47068,19539.17278,7738.11
220,Várzea,2517100.0,varzense,OTONI COSTA DE MEDEIROS,191.282,2870.0,13.15,99.5,0.707,-,10942.50545,11283.73348,11215.80
221,Vieirópolis,2517209.0,vieiropolense,JOSE CELIO ARISTOTELES,147.098,5395.0,34.37,98.6,0.571,-,14497.69028,13410.14001,7827.71


In [13]:
dados_IDH.columns

Index(['Município [-]', 'Código [-]', 'Gentílico [-]', 'Prefeito [2021]',
       'Área Territorial - km² [2021]', 'População estimada - pessoas [2021]',
       'Densidade demográfica - hab/km² [2010]',
       'Escolarização <span>6 a 14 anos</span> - % [2010]',
       'IDHM <span>Índice de desenvolvimento humano municipal</span> [2010]',
       'Mortalidade infantil - óbitos por mil nascidos vivos [2020]',
       'Receitas realizadas - R$ (×1000) [2017]',
       'Despesas empenhadas - R$ (×1000) [2017]',
       'PIB per capita - R$ [2019]'],
      dtype='object')

In [14]:
aux3 = pd.DataFrame(aux)
aux3['Área Territorial - km² [2021]'] = None
aux3['População estimada - pessoas [2021]'] = None
aux3['IDHM <span>Índice de desenvolvimento humano municipal</span> [2010]'] = None
aux3['PIB per capita - R$ [2019]'] = None

aux32 = list()
i=0

for row_pb in aux3.iterrows():
    
    nome_munic = row_pb[1]['Nome_Munic']

    for row in dados_IDH.iterrows():

        if row[1]['Município [-]'] == nome_munic:

            aux3.loc[i,'Área Territorial - km² [2021]'] = row[1]['Área Territorial - km² [2021]']
            aux3.loc[i,'População estimada - pessoas [2021]'] = row[1]['População estimada - pessoas [2021]']
            aux3.loc[i,'IDHM <span>Índice de desenvolvimento humano municipal</span> [2010]'] = row[1]['IDHM <span>Índice de desenvolvimento humano municipal</span> [2010]']
            aux3.loc[i,'PIB per capita - R$ [2019]'] = row[1]['PIB per capita - R$ [2019]']

            i+=1
            break

aux3.head()

,OBJECTID,GEOCODIG_M,UF,Sigla,Nome_Munic,Região,Mesorregiã,Nome_Meso,Microrregi,Nome_Micro,Shape_Leng,Shape_Area,geometry,Prestador,Área Territorial - km² [2021],População estimada - pessoas [2021],IDHM <span>Índice de desenvolvimento humano municipal</span> [2010],PIB per capita - R$ [2019]
0,1,2500106.0,25,PB,Água Branca,Nordeste,2501,Sertão Paraibano,25007,Serra do Teixeira,0.729661,0.019337,"POLYGON ((-37.70376 -7.37318, -37.69232 -7.378...",Prefeitura Municipal de Água Branca,241.662,10375.0,0.572,8074.16
1,2,2500205.0,25,PB,Aguiar,Nordeste,2501,Sertão Paraibano,25005,Piancó,0.969734,0.028152,"POLYGON ((-38.20504 -7.00981, -38.15629 -7.009...",Sem informações,351.607,5622.0,0.597,8897.54
2,3,2500304.0,25,PB,Alagoa Grande,Nordeste,2503,Agreste Paraibano,25015,Brejo Paraibano,0.789787,0.026308,"POLYGON ((-35.53465 -7.00937, -35.52336 -7.031...",Companhia de Águas e Esgotos da Paraíba,322.071,28384.0,0.582,9699.38
3,4,2500403.0,25,PB,Alagoa Nova,Nordeste,2503,Agreste Paraibano,25015,Brejo Paraibano,0.500304,0.009923,"POLYGON ((-35.79566 -7.02333, -35.78838 -7.029...",Sem informações,128.23,20992.0,0.576,15631.55
4,5,2500502.0,25,PB,Alagoinha,Nordeste,2503,Agreste Paraibano,25016,Guarabira,0.444048,0.007825,"POLYGON ((-35.56345 -6.90760, -35.53411 -6.928...",Sem informações,111.361,14629.0,0.595,7934.57


In [15]:
aux3.to_csv('./dados_finalizados/municipios_completo.csv')

In [42]:
cidades_sem_esgoto = aux3[aux3['Prestador'] == 'Sem informações']
cidades_sem_esgoto.to_csv('./dados_finalizados/municipios_sem_rede_com_IDH_PIB.csv')

In [22]:
cidades_adm_prefeitura = aux3[aux3['Prestador'] != 'Sem informações']
cidades_adm_prefeitura = cidades_adm_prefeitura[cidades_adm_prefeitura['Prestador'] != 'Companhia de Águas e Esgotos da Paraíba']
cidades_adm_prefeitura.head()

,OBJECTID,GEOCODIG_M,UF,Sigla,Nome_Munic,Região,Mesorregiã,Nome_Meso,Microrregi,Nome_Micro,Shape_Leng,Shape_Area,geometry,Prestador,Área Territorial - km² [2021],População estimada - pessoas [2021],IDHM <span>Índice de desenvolvimento humano municipal</span> [2010],PIB per capita - R$ [2019]
0,1,2500106.0,25,PB,Água Branca,Nordeste,2501,Sertão Paraibano,25007,Serra do Teixeira,0.729661,0.019337,"POLYGON ((-37.70376 -7.37318, -37.69232 -7.378...",Prefeitura Municipal de Água Branca,241.662,10375.0,0.572,8074.16
9,10,2500734.0,25,PB,Amparo,Nordeste,2502,Borborema,25010,Cariri Ocidental,0.439098,0.010045,"POLYGON ((-36.99898 -7.60599, -36.98189 -7.619...",Prefeitura Municipal de Amparo,122.094,2264.0,0.606,11708.73
10,11,2500775.0,25,PB,Aparecida,Nordeste,2501,Sertão Paraibano,25003,Sousa,0.792825,0.024188,"POLYGON ((-38.01610 -6.67007, -38.01156 -6.674...",Prefeitura Municipal de Aparecida,291.478,8482.0,0.578,10235.09
12,13,2500908.0,25,PB,Arara,Nordeste,2503,Agreste Paraibano,25012,Curimataú Ocidental,0.478629,0.008029,"POLYGON ((-35.72910 -6.81686, -35.71975 -6.824...",Prefeitura Municipal de Arara,91.306,13613.0,0.548,7615.79
15,16,2501153.0,25,PB,Areia de Baraúnas,Nordeste,2501,Sertão Paraibano,25004,Patos,0.427620,0.007825,"POLYGON ((-36.93721 -7.04681, -36.92925 -7.057...",Prefeitura Municipal de Areia de Baraunas,114.078,2105.0,0.562,9669.46


In [23]:
cidades_adm_prefeitura.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 51 entries, 0 to 220
Data columns (total 18 columns):
 #   Column                                                               Non-Null Count  Dtype   
---  ------                                                               --------------  -----   
 0   OBJECTID                                                             51 non-null     int64   
 1   GEOCODIG_M                                                           51 non-null     float64 
 2   UF                                                                   51 non-null     object  
 3   Sigla                                                                51 non-null     object  
 4   Nome_Munic                                                           51 non-null     object  
 5   Região                                                               51 non-null     object  
 6   Mesorregiã                                                           51 non-null     object  
 7   

In [24]:
cidades_adm_prefeitura.to_csv('./dados_finalizados/municipios_prestador_prefeitura_atualizado.csv')

In [ ]:
aux4 = gpd.GeoDataFrame(
    aux,
    crs={'init': 'epsg:4618'},
    geometry= aux['geometry']
)

aux4.to_crs('EPSG:3857')
aux4.to_file('./dados_finalizados/municipios_completo.shp', encoding='utf-8')
aux4.info()

In [20]:
aux3.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 223 entries, 0 to 222
Data columns (total 18 columns):
 #   Column                                                               Non-Null Count  Dtype   
---  ------                                                               --------------  -----   
 0   OBJECTID                                                             223 non-null    int64   
 1   GEOCODIG_M                                                           223 non-null    float64 
 2   UF                                                                   223 non-null    object  
 3   Sigla                                                                223 non-null    object  
 4   Nome_Munic                                                           223 non-null    object  
 5   Região                                                               223 non-null    object  
 6   Mesorregiã                                                           223 non-null    object  
 7  

In [4]:
municipios_completos = pd.read_csv('./dados_finalizados/municipios_completo.csv')

In [5]:
municipios_completos.head()

,Unnamed: 0,OBJECTID,GEOCODIG_M,UF,Sigla,Nome_Munic,Região,Mesorregiã,Nome_Meso,Microrregi,Nome_Micro,Shape_Leng,Shape_Area,geometry,Prestador,Área Territorial - km² [2021],População estimada - pessoas [2021],IDHM <span>Índice de desenvolvimento humano municipal</span> [2010],PIB per capita - R$ [2019]
0,0,1,2500106.0,25,PB,Água Branca,Nordeste,2501,Sertão Paraibano,25007,Serra do Teixeira,0.729661,0.019337,POLYGON ((-37.70375999999851 -7.37317999999868...,Prefeitura Municipal de Água Branca,NaN,NaN,NaN,NaN
1,1,2,2500205.0,25,PB,Aguiar,Nordeste,2501,Sertão Paraibano,25005,Piancó,0.969734,0.028152,POLYGON ((-38.20503999999892 -7.00980999999774...,Sem informações,NaN,NaN,NaN,NaN
2,2,3,2500304.0,25,PB,Alagoa Grande,Nordeste,2503,Agreste Paraibano,25015,Brejo Paraibano,0.789787,0.026308,POLYGON ((-35.53464999999778 -7.00936999999794...,Companhia de Águas e Esgotos da Paraíba,NaN,NaN,NaN,NaN
3,3,4,2500403.0,25,PB,Alagoa Nova,Nordeste,2503,Agreste Paraibano,25015,Brejo Paraibano,0.500304,0.009923,POLYGON ((-35.79565999999795 -7.02332999999816...,Sem informações,NaN,NaN,NaN,NaN
4,4,5,2500502.0,25,PB,Alagoinha,Nordeste,2503,Agreste Paraibano,25016,Guarabira,0.444048,0.007825,POLYGON ((-35.56344999999783 -6.90759999999863...,Sem informações,NaN,NaN,NaN,NaN
